파이썬 머신러닝 완벽 가이드 ch8. 4 pg. 487~496

## 04 텍스트 분류 실습 - 20 뉴스그룹 분류

- 텍스트 분류: 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것

- 텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 됨. 로지스틱 회귀, 선형 SVM, 나이브 베이즈 등이 잘 처리할 수 있음.

### 텍스트 정규화

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all', random_state=156)

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd
print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('\ntarget 클래스의 이름들\n', news_data.target_names)
# Target 클래스는 0~19

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64

target 클래스의 이름들
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [8]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [9]:
from sklearn.datasets import fetch_20newsgroups
# subset='train'으로 학습용 데이터만 추출, remove=('headers','footers','quotes)로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target

test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'), random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

- 테스트 데이터에서 CountVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환해야 함.

- 테스트 데이터의 피처 벡터화 시 fit_transform()을 사용하면 안됨. CounterVectorizer.fit_transform()을 테스트 데이터 세트에 적용하면 테스트 데이터 기반으로 다시 CounterVectorizer가 fit()을 수행하고 transform()하기 때문에 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라짐.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# Count Vectorizeration으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)
# 학습 데이터를 CountVectorizer로 피처를 추출한 결과 11314개의 문서에서 101631개의 단어가 만들어짐.

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


In [11]:
# 피처 벡터화된 데이터에 로지스틱 회귀 적용
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.608


In [12]:
# Count 기반에서 TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


- TF-IDF가 단순 카운트 기반보다 훨씬 높은 정확도를 제공함.

- 일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출함.

- 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은 최적의 ML알고리즘을 선택하는 것과 최상의 피처 전처리(텍스트 정규화, Count/TF-IDF 기반 피처 벡터화를 효과적으로 적용)를 수행하는 것.

In [13]:
# stop words 필터링을 추가하고 ngrams을 기본(1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


In [20]:
# GridSearchCV를 이용해 로지스틱 회귀의 하이퍼파라미터 최적화 수행
# 최적 C값 도출 튜닝 수행. CV는 3폴드 세트로 생성
from sklearn.model_selection import GridSearchCV
params = {'C':[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid = params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter : ', grid_cv_lr.best_params_)

# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))
# 로지스틱 회귀의 C가 10일 때 가장 좋은 예측 성능을 나타냄.

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter :  {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


### 사이킷런 파이프라인 (Pipeline) 사용 및 GridSearchCV와의 결합


사이킷런의 Pipeline 클래스

-  피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행 가능

- 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 마치 수도관에서 물이 흐르듯 한꺼번에 스트림 기반으로 처리한다는 의미

- 텍스트 기반의 피처 벡터화뿐만 아니라 모든 데이터 전처리 작업(스케일링, 벡터 정규화, PCA 등)과 Estimator를 결합할 수 있음.



In [14]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),
                     ('lr_clf', LogisticRegression(random_state=156))])
# TdifdfVectorizer 객체를 tdidf_vect라는 객체 변수 명으로, LogisticRegression 객체를 lr_clf라는 객체 변수 명으로 생성한 뒤 이 2개의 객체를 파이프라인으로 연결하는 Pipeline 객체 pipeline을 생성한다는 의미

In [15]:
from sklearn.pipeline import Pipeline
# TfidfVectorizer 객체를 tdidf_vect로, LogisticRegression객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tdidf_vect',TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])
# 별도의 TdidfVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요 없음.
# pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
         'tfidf_vect__max_df': [100,300,700],
         'lr_clf__C':[1,5,10]}
         
# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
from sklearn.model_selection import GridSearchCV
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
